<a href="https://colab.research.google.com/github/Deepandreinforcement/Zoom-Out-Animation/blob/main/Zoom_out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Buradaki kodları bu GitHub reposundan aldım. Gerekli birkaç değişiklik yaptım kütüphane kurulumu vs. 

https://github.com/hardmaru/image-notebook/blob/main/stable-diffusion-2-infinite-zoom-out/infinite_zoom_out.ipynb

In [ ]:
!pip install diffusers

In [ ]:
!pip install transformers

In [ ]:
!pip install -U xformers

In [ ]:
!pip install accelerate

In [ ]:
import PIL
from PIL import Image
import requests
from io import BytesIO
import numpy as np
import random
import cv2
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler

In [6]:
def write_video(file_path, frames, fps):
    """
    Writes frames to an mp4 video file
    :param file_path: Path to output video, must end with .mp4
    :param frames: List of PIL.Image objects
    :param fps: Desired frame rate
    """

    w, h = frames[0].size
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    #fourcc = cv2.VideoWriter_fourcc(*'avc1')
    writer = cv2.VideoWriter(file_path, fourcc, fps, (w, h))

    for frame in frames:
        np_frame = np.array(frame.convert('RGB'))
        cv_frame = cv2.cvtColor(np_frame, cv2.COLOR_RGB2BGR)
        writer.write(cv_frame)

    writer.release() 

In [ ]:
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-inpainting", torch_dtype=torch.float16, revision="fp16")
pipe.set_use_memory_efficient_attention_xformers(True)

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")
def dummy(images, **kwargs):
    return images, False
pipe.safety_checker = dummy

In [8]:
current_image = PIL.Image.new(mode="RGBA", size=(512, 512))
mask_image = np.array(current_image)[:,:,3] # assume image has alpha mask (use .mode to check for "RGBA")
mask_image = Image.fromarray(255-mask_image).convert("RGB")
current_image = current_image.convert("RGB")

In [9]:
# Bu promt kodu oluşturan kişinin yazdığı prompt
# orig_prompt = "Create a relaxing atmosphere with the use of plants and other natural elements. Such as a hanging terrarium or a wall-mounted planter. Include plenty of storage options to keep the space organized and clutter-free. Consider adding a vanity with double sinks and plenty of drawers and cabinets. As well as a wall mounted medicine and towel storage."
# Burada oluşturmasını istediğimiz şeyi tarif ediyoruz.
orig_prompt= 'a futuristic city with robots, cinematic, artistic '
# burada resimde olmasını istemediğimiz şeyleri yazıyoruz.
orig_negative_prompt = "lurry, bad art, blurred, text, watermark"


num_images = 1
prompt = [orig_prompt] * num_images
negative_prompt = [orig_negative_prompt] * num_images

In [ ]:
# Burada başlangıç resmini oluşturyoruz.

images = pipe(prompt=prompt, negative_prompt=negative_prompt, image=current_image, mask_image=mask_image, num_inference_steps=25)[0]
current_image = images[0]

current_image

In [11]:
# Burada toplam 512 oluşturdum ben. İlk görüntü yukarıda oluşturulduğu için NUMFRAMES = 511 olarak yazdım.
STEPSIZE = 32
NUMFRAMES = 511
all_frames = []
all_frames.append(current_image)

In [ ]:
for i in range(NUMFRAMES):
  next_image = np.array(current_image.convert("RGBA"))*0
  prev_image = current_image.resize((512-2*STEPSIZE,512-2*STEPSIZE))
  prev_image = prev_image.convert("RGBA")
  prev_image = np.array(prev_image)
  next_image[:, :, 3] = 1
  next_image[STEPSIZE:512-STEPSIZE,STEPSIZE:512-STEPSIZE,:] = prev_image
  prev_image = Image.fromarray(next_image)
  current_image = prev_image
  mask_image = np.array(current_image)[:,:,3] # assume image has alpha mask (use .mode to check for "RGBA")
  mask_image = Image.fromarray(255-mask_image).convert("RGB")
  current_image = current_image.convert("RGB")
  images = pipe(prompt=prompt, negative_prompt=negative_prompt, image=current_image, mask_image=mask_image, num_inference_steps=25)[0]
  current_image = images[0]
  current_image.paste(prev_image, mask=prev_image)
  all_frames.append(current_image)

In [13]:
# Burada oluşturulan videoya isim veriyoruz. Ayrıca FPS'i 16 olarak ayarladım ben.
write_video("infinite_zoom_out.mp4", all_frames, 16)